In [5]:
%run generate_new_examples.ipynb     # importing functions

[]


In [6]:
# jsonfilename = "noname.json"
limits = [0.05, 0.005, 0.0010, 0.0005, 0.0001]

# Rounding detection

This notebook's purpose is to test a following hypothesis: given a "cube" defined by real-valued coordinates from the cube algorithm, assume a rounding of these coordinates, which are in some sense *nearly* an integer. We test whether this rounding gives us the desired coordinate of the linear combination given by the LLL. 

Note: this may not work always, as sometimes the jump is greater than 1.

In [84]:
def detect_rounding(jsonfilename):
    '''
    OUTPUT: dict in the form \{ limit: [succesful attempts, unsuc att.],...\}
    '''
    cases = from_json(jsonfilename)
    stats = {}
    for limit in limits:
        stats[limit] = [0, 0,0,0 ] 
    for case in cases:
        for i in range(len(case["lincomb_cube"])):
            
            #skip equal components
            if case["lincomb_diff"][i] == 0:
                continue
                
            for limit in limits:
                num_cube = case["lincomb_cube"][i]
                num_LLL = case["lincomb_LLL"][i]
                is_near_int, direction =  is_nearest_integer(num_cube, limit)
                if is_near_int: # if its almost an int
                    if round(num_cube) == num_LLL: 
                        stats[limit] = increment(stats, limit, 0)                        # rounding lead directly to lcLLL
                    else:
                        stats[limit] = increment(stats, limit, 1)                        # rounding didnt lead directly to lcLLL
                        if lcLLL_is_in_same_direction(direction, num_cube, num_LLL):
                            stats[limit] = increment(stats, limit, 2)                    # but it was in the same direction
                        else: stats[limit] = increment(stats, limit, 3)
    return stats

def is_nearest_integer(num, tolerance):
    nearest_int = round(num)
    is_within_tolerance = abs(num - nearest_int) <= tolerance
    if not is_within_tolerance:
        return False, None
    direction = "up" if num - nearest_int < 0 else "down"
    return True, direction


def increment(dic, limit, ind):
    lst = [dic.get(limit,0)[0], dic.get(limit,0)[1],  dic.get(limit,0)[2],  dic.get(limit,0)[3]]
    lst[ind] += 1
    return lst

def lcLLL_is_in_same_direction(direction, num_cube, num_LLL, ):
    return abs(num_cube - num_LLL) - abs(round(num_cube) - num_LLL) > 0


In [85]:
for [limit, [a,b,c,d]] in detect_rounding("matrices6x6-12000instances.json").items():
    print("success: {}  fail: {} for limit = {}".format(a,b, limit.n(digits=1)))
    print("successful rounding ratio: ", 100 *a/(a + b).n(digits=3), " % out of all nearly integers")
    print()
    
for [limit, [a,b,c,d]] in detect_rounding("matrices6x6-12000instances.json").items():
    print(limit, a, b,c,d)

success: 338  fail: 2430 for limit = 0.050
successful rounding ratio:  12.2  % out of all nearly integers

success: 26  fail: 220 for limit = 0.0050
successful rounding ratio:  10.6  % out of all nearly integers

success: 8  fail: 44 for limit = 0.0010
successful rounding ratio:  15.4  % out of all nearly integers

success: 1  fail: 20 for limit = 0.00050
successful rounding ratio:  4.76  % out of all nearly integers

success: 0  fail: 6 for limit = 0.00010
successful rounding ratio:  0.000  % out of all nearly integers

0.0500000000000000 338 2430 1192 1238
0.00500000000000000 26 220 118 102
0.00100000000000000 8 44 21 23
0.000500000000000000 1 20 9 11
0.000100000000000000 0 6 2 4


In most cases, the success rate is about 10-15 \%, which is quite surprising. I thought that as the limit gets lower, fewer cases would success and the success rate would decrease. 

## Not wall, at least direction?


So now, we've 